#Exercise 7

In [15]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X_moons, y_moons = make_moons(n_samples=1000, noise=0.4, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_moons,y_moons, test_size=0.2)

In [16]:
X_train.shape

(800, 2)

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

clf = DecisionTreeClassifier(random_state=42)
params = {
    "max_leaf_nodes":[1,5,10]
}
grid_search = GridSearchCV(clf,params,cv=5)
grid_search.fit(X_train, y_train)


/Users/lucaswerner/Desktop/hands-on-machine-learning/.env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
5 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/lucaswerner/Desktop/hands-on-machine-learning/.env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/lucaswerner/Desktop/hands-on-machine-learning/.env/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 969, in fit
    super().fit(
  File "/Users/lucaswerner/Desktop/hands-on-machine-learning/.env/lib/p

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_leaf_nodes': [1, 5, 10]})

In [36]:
grid_search.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=5, random_state=42)

In [20]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, grid_search.predict(X_test))

0.845

# Exercise 8

In [35]:
from sklearn.model_selection import ShuffleSplit
n_splits = 1000
rs = ShuffleSplit(n_splits, train_size=100,random_state=42)

In [43]:
grid_search.best_estimator_.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': 5,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 42,
 'splitter': 'best'}

In [ ]:
clf_estimators = []
clf_scores = []
for i, (train_index, test_index) in enumerate(rs.split(X_train)):
    x,y = X_train[train_index], y_train[train_index]
    clf = DecisionTreeClassifier(**grid_search.best_estimator_.get_params())
    clf.fit(x,y)
    score = accuracy_score(y_test, clf.predict(X_test))
    clf_scores.append(score)
    clf_estimators.append(clf)

In [48]:
import numpy as np

clf_scores = np.array(clf_scores)

In [51]:
from scipy import stats

stats.mode(clf_scores)[0]

/var/folders/m3/_my7yl913gs59vc5prp0lrzh0000gn/T/ipykernel_81047/1519911690.py:3: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  stats.mode(clf_scores)[0]


array([0.845])

In [62]:
forest_predictions = []
for i,x in enumerate(X_test):
	forest_predictions.append(np.array([clf.predict([x])[0] for clf in clf_estimators], dtype=np.uint0))

In [63]:
forest_predictions = np.array(forest_predictions, dtype=np.uint8)
forest_predictions.shape

(200, 1000)

In [71]:
accuracy_score(y_test, stats.mode(forest_predictions,axis=1)[0].ravel())

/var/folders/m3/_my7yl913gs59vc5prp0lrzh0000gn/T/ipykernel_81047/687905897.py:1: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  accuracy_score(y_test, stats.mode(forest_predictions,axis=1)[0].ravel())


0.86